# Data Wrangling - Historic NFL QB Data 

In [94]:
import pandas as pd
import numpy as np
import os
from os.path import dirname

In [129]:
cwd = os.getcwd()
d = dirname(cwd)
f = "Data_files"
file = "Game_Logs_Quarterback.csv"
path = os.path.join(d, f, file)
df = pd.read_csv(path)
df.head()

,Player Id,Name,Position,Year,Season,Week,Game Date,Home or Away,Opponent,Outcome,...,Ints,Sacks,Sacked Yards Lost,Passer Rating,Rushing Attempts,Rushing Yards,Yards Per Carry,Rushing TDs,Fumbles,Fumbles Lost
0,jaredzabransky/2495791,"Zabransky, Jared",NaN,2007,Preseason,1,11-Aug,Home,CHI,L,...,--,--,--,0.0,--,--,--,--,--,--
1,jaredzabransky/2495791,"Zabransky, Jared",NaN,2007,Preseason,2,18-Aug,Away,ARI,W,...,0,0,0,46.9,--,--,--,--,--,--
2,jaredzabransky/2495791,"Zabransky, Jared",NaN,2007,Preseason,3,25-Aug,Home,DAL,W,...,--,--,--,0.0,--,--,--,--,--,--
3,jaredzabransky/2495791,"Zabransky, Jared",NaN,2007,Preseason,4,30-Aug,Away,TB,L,...,1,2,13,50.8,2,17,8.5,0,--,--
4,billdemory/2512778,"Demory, Bill",NaN,1974,Regular Season,1,15-Sep,Away,KC,L,...,--,--,--,0.0,--,--,--,--,--,--


In [128]:
df.columns.tolist()

['Player Id',
 'Name',
 'Position',
 'Year',
 'Season',
 'Week',
 'Game Date',
 'Home or Away',
 'Opponent',
 'Outcome',
 'Score',
 'Games Played',
 'Games Started',
 'Passes Completed',
 'Passes Attempted',
 'Completion Percentage',
 'Passing Yards',
 'Passing Yards Per Attempt',
 'TD Passes',
 'Ints',
 'Sacks',
 'Sacked Yards Lost',
 'Passer Rating',
 'Rushing Attempts',
 'Rushing Yards',
 'Yards Per Carry',
 'Rushing TDs',
 'Fumbles',
 'Fumbles Lost']

In [134]:
QB_data = pd.DataFrame(df[['Name','Year','Season','Week','Outcome','Passing Yards','TD Passes']])
QB_data.head()

,Name,Year,Season,Week,Outcome,Passing Yards,TD Passes
0,"Zabransky, Jared",2007,Preseason,1,L,--,--
1,"Zabransky, Jared",2007,Preseason,2,W,19,0
2,"Zabransky, Jared",2007,Preseason,3,W,--,--
3,"Zabransky, Jared",2007,Preseason,4,L,117,0
4,"Demory, Bill",1974,Regular Season,1,L,--,--


In [135]:
QB_data.dtypes

Name             object
Year              int64
Season           object
Week              int64
Outcome          object
Passing Yards    object
TD Passes        object
dtype: object

# Clean data

In [136]:
QB_data.replace("--",0, inplace=True)
QB_data['Passing Yards']= QB_data['Passing Yards'].astype('int')
QB_data['TD Passes']= QB_data['TD Passes'].astype('int')
QB_data.dtypes

Name             object
Year              int64
Season           object
Week              int64
Outcome          object
Passing Yards     int32
TD Passes         int32
dtype: object

In [137]:
def wins(row):
    if row['Outcome'] == 'W':
        val = 1
    else:
        val = 0
    return val

In [138]:
QB_data['Wins'] = QB_data.apply(wins, axis=1)
QB_data.head()

,Name,Year,Season,Week,Outcome,Passing Yards,TD Passes,Wins
0,"Zabransky, Jared",2007,Preseason,1,L,0,0,0
1,"Zabransky, Jared",2007,Preseason,2,W,19,0,1
2,"Zabransky, Jared",2007,Preseason,3,W,0,0,1
3,"Zabransky, Jared",2007,Preseason,4,L,117,0,0
4,"Demory, Bill",1974,Regular Season,1,L,0,0,0


In [179]:
def abbr_name(row):
    full_name = row['Name']
    last_name = full_name.split()[0]
    last_name = last_name.replace(',', '')
    first_name = full_name.split()[1]
    first_nam_initial = first_name[:1]
    abbreviated_name = first_nam_initial + "." + last_name
    return abbreviated_name

In [180]:
QB_data['Abbr_Name'] = QB_data.apply(abbr_name, axis=1)
QB_data.head()

,Name,Year,Season,Week,Outcome,Passing Yards,TD Passes,Wins,Abbr_Name
0,"Zabransky, Jared",2007,Preseason,1,L,0,0,0,J.Zabransky
1,"Zabransky, Jared",2007,Preseason,2,W,19,0,1,J.Zabransky
2,"Zabransky, Jared",2007,Preseason,3,W,0,0,1,J.Zabransky
3,"Zabransky, Jared",2007,Preseason,4,L,117,0,0,J.Zabransky
4,"Demory, Bill",1974,Regular Season,1,L,0,0,0,B.Demory


# Group data

In [181]:
QB_data_groups = QB_data.groupby(['Year','Abbr_Name','Name','Season'])['TD Passes','Passing Yards','Wins'].sum()
QB_data_groups = QB_data_groups.reset_index()
QB_data_groups.head()

,Year,Abbr_Name,Name,Season,TD Passes,Passing Yards,Wins
0,1970,A.Pastrana,"Pastrana, Al",Regular Season,1,371,1
1,1970,B.Belden,"Belden, Bob",Regular Season,0,0,2
2,1970,B.Cappleman,"Cappleman, Bill",Regular Season,0,49,2
3,1970,B.Davis,"Davis, Bob",Regular Season,0,66,3
4,1970,B.Kilmer,"Kilmer, Billy",Regular Season,4,1447,2


In [182]:
# Save cleaned data to csv new file
QB_data_groups.to_csv("C:/Users/May 2018/Desktop/SMU Bootcamp Doc/01-Homework_Files/NFL-QB-Analytics/Assets/Data_Files/Cleaned_QB_data_groups.csv")

In [183]:
# Save cleaned data to csv new file
QB_data.to_csv("C:/Users/May 2018/Desktop/SMU Bootcamp Doc/01-Homework_Files/NFL-QB-Analytics/Assets/Data_Files/Cleaned_QB_data.csv")

In [184]:
QB_data_groups_T = QB_data_groups.transpose()
QB_data_groups_T
QB_data_groups_T.to_csv("C:/Users/May 2018/Desktop/SMU Bootcamp Doc/01-Homework_Files/NFL-QB-Analytics/Assets/Data_Files/Cleaned_QB_data_group_transposed.csv")

In [185]:
#Dependencies
import sqlalchemy
from sqlalchemy import create_engine, func, inspect
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

# PyMySQL
import pymysql
import pymysql.cursors
pymysql.install_as_MySQLdb()

In [186]:
start = "mysql://root:"
end = "@localhost:3306/qb_data"
password ="lsu671234"
path = start + password + end
engine = create_engine(path)
#"mysql://root: @localhost:3306/etl_project_db"

In [187]:
QB_data_groups.to_sql(name='qb_data4', con=engine, if_exists='replace', index=False)

In [188]:
QBSQL = engine.execute("SELECT * FROM QB_data4").fetchall()
QBSQL = pd.DataFrame(QBSQL)
QBSQL.head()

,0,1,2,3,4,5,6
0,1970,A.Pastrana,"Pastrana, Al",Regular Season,1,371,1
1,1970,B.Belden,"Belden, Bob",Regular Season,0,0,2
2,1970,B.Cappleman,"Cappleman, Bill",Regular Season,0,49,2
3,1970,B.Davis,"Davis, Bob",Regular Season,0,66,3
4,1970,B.Kilmer,"Kilmer, Billy",Regular Season,4,1447,2
